<a href="https://colab.research.google.com/github/justinballas/Finance-Projects/blob/main/historical_change_pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime as dt
import requests
api_key = ''

In [ ]:
#https://financialmodelingprep.com/api/v3/profile/AAPL?apikey=YOUR_API_KEY
data = requests.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/AAPL?apikey={api_key}').json()

In [ ]:
df = pd.DataFrame(data['historical'])
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [ ]:
def pct_change_pivot(data, period='year', average=False):
  def quarter(date):
    '''
    This function takes in a series of dates, then converts them to month and day
    values, to be converted into fiscal quarters.
    '''
    date=date.strftime('%m-%d')

    if date >='01-01' and date <= '03-31':
      return 1
    elif date >= '04-01' and date <= '06-31':
      return 2
    elif date >= '07-01' and date <= '09-30':
      return 3
    else:
      return 4

  temp_df = pd.DataFrame(data)
  temp_df['Quarter'] = list(pd.Series(df.index).apply(lambda x: quarter(x)))
  temp_df['Month'] = list(pd.Series(temp_df.index).apply(lambda x: x.month))
  temp_df['Year'] = list(pd.Series(temp_df.index).apply(lambda x: x.year))
  temp_df['Quarter_diff'] = temp_df['Quarter'].diff()
  temp_df['Month_diff'] = temp_df['Month'].diff()
  temp_df['Year_diff'] = temp_df['Year'].diff()

  if period=='year':
    pivot_table = pd.DataFrame(temp_df[temp_df['Year_diff'] != 0]['adjClose'])[::-1].pct_change()[::-1]
  elif period=='month':
    pivot_table = pd.DataFrame(temp_df[temp_df['Month_diff'] != 0]['adjClose'])[::-1].pct_change()[::-1]
  elif period=='quarter':
    pivot_table = pd.DataFrame(temp_df[temp_df['Quarter_diff'] != 0]['adjClose'])[::-1].pct_change()[::-1]
  else:
    pass

  total_table = pd.merge(df2, df3, how='left', left_index=True, right_index=True)
  
  return pivot_table, temp_df

In [ ]:
pd.set_option('display.max_rows', 25)
df2, df3 = pct_change_pivot(df['adjClose'], period='year')

In [ ]:
#Notes For Averaging
pd.merge(df2, df3, how='left', left_index=True, right_index=True).groupby('Year').mean()['adjClose_x'].sort_values(ascending=False)

Year
2019    0.889578
2020    0.823067
2017    0.484643
2021    0.144246
2018   -0.053902
2016         NaN
Name: adjClose_x, dtype: float64

Next steps:
  Create quarter and possibly hour/minute options
  Turn index into date range
  Create functionality to have multiple time periods/multi index
  Add option to have a simplified naming convention (month name with year etc.)